<a href="https://colab.research.google.com/github/SreyaJampana/Cognitive-Distortion-Detection/blob/main/DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from google.colab import files
uploaded = files.upload()

# Load the dataset
annotated_data = pd.read_csv('Annotated_data.csv')

# Fill missing labels for "No distortion" to standardize the data
annotated_data['Dominant Distortion'].fillna('No distortion', inplace=True)

# Preprocessing function to encode labels
def encode_labels(labels):
    unique_labels = sorted(set(labels))
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
    id_to_label = {idx: label for label, idx in label_to_id.items()}
    return [label_to_id[label] for label in labels], label_to_id, id_to_label

# Encode labels
encoded_labels, label_to_id, id_to_label = encode_labels(annotated_data['Dominant Distortion'])

# Add encoded labels to the dataset
annotated_data['encoded_label'] = encoded_labels

# Split dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    annotated_data['Patient Question'].tolist(),
    annotated_data['encoded_label'].tolist(),
    test_size=0.2, random_state=42
)

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenization function
def tokenize_data(texts, labels, max_length=128):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
    encodings['labels'] = labels
    return encodings

# Tokenize train and test sets
train_encodings = tokenize_data(train_texts, train_labels)
test_encodings = tokenize_data(test_texts, test_labels)

# Create custom Dataset class
class CognitiveDistortionDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

# Create PyTorch Dataset objects
train_dataset = CognitiveDistortionDataset(train_encodings)
test_dataset = CognitiveDistortionDataset(test_encodings)

# Load datasets into DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Load the pre-trained DistilBERT model for classification
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_to_id)
)
model.to(device)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * 4  # Assuming 4 epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# Training loop
def train_model(model, train_loader, optimizer, scheduler, device, epochs=4):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Backward pass
            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

# Train the model
train_model(model, train_loader, optimizer, scheduler, device, epochs=4)

# Evaluate the model
preds, true_labels = evaluate_model(model, test_loader, device)

# Report performance
print("Accuracy:", accuracy_score(true_labels, preds))
print(classification_report(true_labels, preds, target_names=list(label_to_id.keys())))


Saving Annotated_data.csv to Annotated_data.csv


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4, Loss: 2.0607
Epoch 2/4, Loss: 1.9575
Epoch 3/4, Loss: 1.8011
Epoch 4/4, Loss: 1.5759
Accuracy: 0.35177865612648224
                         precision    recall  f1-score   support

All-or-nothing thinking       0.00      0.00      0.00        28
    Emotional Reasoning       0.33      0.04      0.06        28
        Fortune-telling       0.10      0.10      0.10        31
               Labeling       0.08      0.07      0.08        27
          Magnification       0.18      0.26      0.21        43
          Mental filter       0.19      0.17      0.18        30
           Mind Reading       0.23      0.30      0.26        47
          No Distortion       0.56      0.76      0.65       175
     Overgeneralization       0.14      0.20      0.17        46
        Personalization       0.00      0.00      0.00        32
      Should statements       0.00      0.00      0.00        19

               accuracy                           0.35       506
              macro avg    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
